<center style="font-size:48px;">Clean Up</center>
<br>
Steps needed to clean the data and create useful features to use in our analysis

# Importing Libraries and Data

## Libraries

In [46]:
# Data Science
import pandas as pd 

## Data

In [47]:
cars = pd.read_csv('../Data/car-assignments.csv')
cc = pd.read_csv('../Data/cc_data.csv', encoding='cp1252', parse_dates=['timestamp'])
gps = pd.read_csv('../Data/gps.csv', parse_dates=['Timestamp'])
loyalty = pd.read_csv('../Data/loyalty_data.csv', encoding='cp1252', parse_dates=['timestamp'])

# Helper Functions

In [48]:
def isWeekend(x):
    # Finds if a day is a weekend
    if x.weekday() >= 5:
        return True
    else:
        return False

def seperateTimeUnits(df, col):
    # Lets Seperate the variable time units into their own features
    units = ['day', 'hour', 'minute', 'second']
    for unit in units:
        if unit == 'day':
            df[unit] = df[col].apply(lambda x: x.day)
        if unit == 'hour':
            df[unit] = df[col].apply(lambda x: x.hour)
        if unit == 'minute':
            df[unit] = df[col].apply(lambda x: x.minute)
        if unit == 'second':
            df[unit] = df[col].apply(lambda x: x.second)
    return df

# Cleaning the Data

## Location Data

Merge the gps and the car assignment data frames. Do a left outer join with gos being the left dataframe. This will keep the gps points for the truck drivers. 

In [49]:
locations = gps.merge(cars, left_on='id', right_on='CarID', how='left')
locations.drop(columns='CarID', inplace =True)
locations.head()

,Timestamp,id,lat,long,LastName,FirstName,CurrentEmploymentType,CurrentEmploymentTitle
0,2014-01-06 06:28:01,35,36.076225,24.874689,Vasco-Pais,Willem,Executive,Environmental Safety Advisor
1,2014-01-06 06:28:01,35,36.076220,24.874596,Vasco-Pais,Willem,Executive,Environmental Safety Advisor
2,2014-01-06 06:28:03,35,36.076211,24.874443,Vasco-Pais,Willem,Executive,Environmental Safety Advisor
3,2014-01-06 06:28:05,35,36.076217,24.874253,Vasco-Pais,Willem,Executive,Environmental Safety Advisor
4,2014-01-06 06:28:06,35,36.076214,24.874167,Vasco-Pais,Willem,Executive,Environmental Safety Advisor


For the gps data for the truck drivers the names are null. Lets fill that with the name "Truck Driver_X" where "X" is the CarID number

In [50]:
locations['CurrentEmploymentType'] = locations['CurrentEmploymentType'].fillna('Facilities')
locations['CurrentEmploymentTitle'] = locations['CurrentEmploymentTitle'].fillna('Truck Driver')
locations['LastName'] = locations['LastName'].fillna('Driver')
locations['FirstName'] = locations['FirstName'].fillna('Truck')
locations['LastName'] = locations.apply(lambda x: 'Driver_{}'.format(x['id']) if x['LastName'] == 'Driver' else x['LastName'], axis =1 )

Lets Seperate the varibale time units into their own features

In [51]:
locations = seperateTimeUnits(locations, 'Timestamp')

To shrink data size (so we can put it on github) downsample the data to 1minute, take the median of any duplicate location data (by person and time).

In [52]:
locations = locations.groupby(['FirstName', 'LastName', 'CurrentEmploymentType','CurrentEmploymentTitle']) \
    .resample(rule='1min', on='Timestamp') \
    .median() \
    .reset_index() \
    .dropna() \
    .drop(columns = 'second')

Lets create a feature denoting if a day is the weekend

In [53]:
locations['Weekend'] = locations.apply(lambda x: isWeekend(x['Timestamp']), axis = 1)
locations.sample(10)

,FirstName,LastName,CurrentEmploymentType,CurrentEmploymentTitle,Timestamp,id,lat,long,day,hour,minute,Weekend
449127,Loreto,Bodrogi,Security,Site Control,2014-01-09 19:25:00,15.0,36.058395,24.904370,9.0,19.0,25.0,False
49037,Axel,Calzas,Engineering,Drill Technician,2014-01-13 08:21:00,9.0,36.081730,24.852523,13.0,8.0,21.0,False
2908,Ada,Campo-Corrente,Executive,SVP/CIO,2014-01-08 07:25:00,10.0,36.071693,24.864269,8.0,7.0,25.0,False
507983,Minke,Mies,Security,Perimeter Control,2014-01-10 07:54:00,24.0,36.054492,24.900348,10.0,7.0,54.0,False
588529,Sven,Flecha,Information Technology,IT Technician,2014-01-09 13:47:00,17.0,36.051930,24.878816,9.0,13.0,47.0,False
126894,Edvard,Vann,Security,Perimeter Control,2014-01-13 07:06:00,34.0,36.063750,24.903746,13.0,7.0,6.0,False
535021,Nils,Calixto,Information Technology,IT Technician,2014-01-15 13:29:00,8.0,36.056660,24.863453,15.0,13.0,29.0,False
388831,Lars,Azada,Engineering,Engineer,2014-01-07 08:30:00,2.0,36.068014,24.876308,7.0,8.0,30.0,False
436662,Linnea,Bergen,Information Technology,IT Group Manager,2014-01-13 11:47:00,6.0,36.052706,24.876488,13.0,11.0,47.0,False
315774,Isande,Borrasca,Engineering,Drill Technician,2014-01-10 11:05:00,7.0,36.064023,24.893496,10.0,11.0,5.0,False


This is now a useable set of data for our analysis. We can subset by Name, Job Title, Job Type, Car, day, time of day, and weekend. We can also resample to get a datapoint per 5, 10, 15, etc. minute intervals for ur desired subset.

## Purchase Data

Create a Is Loyalty variable that denotes if a purchase is a loyalty card purchase or not.

In [54]:
cc['Is_Loyalty'] = False
loyalty['Is_Loyalty'] = True

Similar to the locaton data lets seperate the time units and create a weekend variable.

In [55]:
# Lets Seperate the varibale time units into their own features
cc = seperateTimeUnits(cc, 'timestamp')
loyalty = seperateTimeUnits(loyalty, 'timestamp')
# Deontate Weekends
cc['Weekend'] = cc.apply(lambda x: isWeekend(x['timestamp']), axis = 1)
loyalty['Weekend'] = loyalty.apply(lambda x: isWeekend(x['timestamp']), axis = 1)

We can find which transactions are duplicates (at least most of them) by a person's name, purchase location, purchase day, and the cents in the price. The loyalty purchase price is less prone to outliers so we will use that as the final purchase price. The timestamp in the credit card dataframe has more infirmation on time of purchase so we will use that.

In [56]:
# Seperate the cents from the purcahase price
cc['cents'] = round(cc.apply(lambda x: (x['price'] % 1) * 100,  axis = 1))
loyalty['cents'] = round(loyalty.apply(lambda x: (x['price'] % 1) * 100,  axis = 1))
# Loop to match the duplicate purchases. Overwrite the CREDIT CARD dataframe with tthe loyalty price and loyalty card flag values
for index, row in cc.iterrows():
    first = row['FirstName']
    last = row['LastName']
    location = row['location']
    day = row['day']
    cents = row['cents']
    temp = loyalty[loyalty.FirstName == first]
    temp = temp[temp.LastName == last]
    temp = temp[temp.location == location]
    temp = temp[temp.day == day]
    temp = temp[temp.cents == cents]
    if len(temp) >= 1:
        cc.loc[index, 'Is_Loyalty'] = True
        cc.loc[index, 'price'] = temp.price.values

# Merge the two dataframe by appending one to the other and dropping duplicates
buys = pd.concat([cc, loyalty]).drop_duplicates(['FirstName', 'LastName', 'location', 'day', 'cents'], keep='first')
buys.drop(columns  ='cents', inplace =True)

Lets also add the personal information (job title and type). Use a left outer join to not lose data for anybody who isn't in the cars dataframe

In [57]:
buys = buys.merge(cars, left_on=['LastName', 'FirstName'], right_on=['LastName', 'FirstName'], how= 'left')

Replace the null values for job type and title with 'Other'. Also, save their carId as 100

In [58]:
buys.fillna({'CurrentEmploymentType' : 'Other', 'CurrentEmploymentTitle':'Other'}, inplace= True)
def CarNulls(x):
    if pd.isna(x['CarID']):
        if x['CurrentEmploymentTitle'] == 'Truck Driver':
            return 100
        else:
            return 0
    else:
        return x['CarID']

buys['CarID'] = buys.apply(lambda x: CarNulls(x), axis =1)
buys.sample(10)


,timestamp,location,price,FirstName,LastName,Is_Loyalty,day,hour,minute,second,Weekend,CarID,CurrentEmploymentType,CurrentEmploymentTitle
877,2014-01-14 07:35:00,Brew've Been Served,7.31,Isia,Vann,True,14,7,35,0,False,16.0,Security,Perimeter Control
220,2014-01-07 14:34:00,Carlyle Chemical Inc.,4803.13,Valeria,Morlun,False,7,14,34,0,False,100.0,Facilities,Truck Driver
649,2014-01-11 13:51:00,Abila Zacharo,32.02,Linnea,Bergen,False,11,13,51,0,True,6.0,Information Technology,IT Group Manager
1418,2014-01-18 19:35:00,Hippokampos,27.21,Vira,Frente,True,18,19,35,0,True,19.0,Engineering,Hydraulic Technician
27,2014-01-06 08:16:00,Brew've Been Served,11.20,Linda,Lagos,True,6,8,16,0,False,0.0,Other,Other
130,2014-01-07 07:33:00,Brew've Been Served,7.09,Edvard,Vann,False,7,7,33,0,False,34.0,Security,Perimeter Control
1310,2014-01-17 13:33:00,Gelatogalore,23.23,Felix,Balas,True,17,13,33,0,False,3.0,Engineering,Engineer
247,2014-01-07 20:34:00,Hippokampos,10.19,Gustav,Cazar,True,7,20,34,0,False,11.0,Engineering,Hydraulic Technician
1234,2014-01-16 20:13:00,Hippokampos,29.13,Gustav,Cazar,True,16,20,13,0,False,11.0,Engineering,Hydraulic Technician
36,2014-01-06 12:00:00,Bean There Done That,10.28,Lars,Azada,True,6,12,0,0,False,2.0,Engineering,Engineer


# Save the transformed data

In [59]:
locations.to_csv('../CheckPoints/Locations_Clean.csv')
buys.to_csv('../CheckPoints/Buys_Clean.csv')

<div>
    <span  style="width:600px;display:inline-block;text-align:left">
        <a href="./FurtherEDA.ipynb">&#60;&#60;Further Exploratoy Data Analysis</a>
    </span>
    <span style="width:600px;display:inline-block;text-align:right">
        <a href="./Modeling.ipynb">Modeling&#62;&#62;</a>
    </span>
</div>
<div>
    <center>
        <span style="width:200px;display:inline-block;text-align:center">
            <a href="./Master.ipynb">Master Notebook</a>
        </span>
        <span style="width:200px;display:inline-block;text-align:center">
            <a href="../README.md">Table of Contents</a>
        </span>
    </center>
</div>